<a href="https://colab.research.google.com/github/thevedantt/GlucoGuard-AI/blob/main/Diabetes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Diabetes Prediction with AI Recommendations (Updated)
import pandas as pd
import numpy as np
import warnings
import requests
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

warnings.filterwarnings('ignore')

try:
    from colorama import init, Fore, Style
    init(autoreset=True)
    COLOR = True
except ImportError:
    COLOR = False

class DiabetesPredictionSystem:
    def __init__(self, csv_path='diabetes.csv'):
        self.csv_path = csv_path
        self.model = RandomForestClassifier(random_state=42)
        self.gemini_api_key = ""

    def load_and_train_model(self):
        """Load dataset and train model"""
        print("[INFO] Loading data and training model...")

        data = pd.read_csv(self.csv_path)
        data = data.apply(pd.to_numeric, errors='coerce')
        data.dropna(inplace=True)

        X = data.drop('Outcome', axis=1)
        y = data['Outcome']

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        self.model.fit(X_train, y_train)

        accuracy = self.model.score(X_test, y_test)
        print(f"[INFO] Model trained with accuracy: {accuracy:.3f}")

    def get_user_input(self):
        """Collect user input"""
        print("\n" + "="*50)
        print("DIABETES PREDICTION SYSTEM")
        print("="*50)

        while True:
            gender = input("Enter gender (M/F): ").upper()
            if gender in ['M', 'F']:
                break
            print("Please enter 'M' or 'F'")

        user_data = {}
        user_data['Pregnancies'] = int(input("Number of pregnancies (0 if male): ")) if gender == 'F' else 0
        user_data['Glucose'] = float(input("Glucose level (mg/dL): "))
        user_data['BloodPressure'] = float(input("Blood Pressure (mm Hg): "))
        user_data['SkinThickness'] = float(input("Skin Thickness (mm): "))
        user_data['Insulin'] = float(input("Insulin level (mu U/ml): "))
        user_data['BMI'] = float(input("BMI (kg/m²): "))
        user_data['DiabetesPedigreeFunction'] = float(input("Diabetes Pedigree Function: "))
        user_data['Age'] = int(input("Age (years): "))

        return user_data, gender

    def make_prediction(self, user_data):
        """Predict diabetes risk"""
        user_df = pd.DataFrame([user_data])
        prediction = self.model.predict(user_df)[0]
        probability = self.model.predict_proba(user_df)[0]
        return prediction, probability

    def get_ai_recommendations(self, user_data, gender, prediction, probability):
        """Get AI recommendations from Gemini"""
        try:
            risk_level = "HIGH RISK" if prediction == 1 else "LOW RISK"
            diabetes_probability = probability[1] * 100
            confidence = max(probability) * 100

            risk_factors = []
            if user_data['Glucose'] > 140: risk_factors.append("High glucose level")
            if user_data['BMI'] > 30: risk_factors.append("Obesity")
            if user_data['Age'] > 45: risk_factors.append("Older age")
            if user_data['BloodPressure'] > 130: risk_factors.append("High blood pressure")

            prompt = f"""
            You are a diabetes medical assistant. Given this patient profile:

            Gender: {gender}
            Age: {user_data['Age']}
            BMI: {user_data['BMI']}
            Glucose: {user_data['Glucose']}
            Blood Pressure: {user_data['BloodPressure']}
            Insulin: {user_data['Insulin']}
            Pregnancies: {user_data['Pregnancies']}

            DIABETES RISK: {risk_level}
            CONFIDENCE: {confidence:.1f}%
            RISK FACTORS: {', '.join(risk_factors) or 'None'}

            Provide detailed recommendations in 7 sections:
            1. IMMEDIATE ACTIONS
            2. DIETARY PLAN
            3. EXERCISE
            4. LIFESTYLE
            5. MONITORING
            6. FOLLOW-UP
            7. LONG-TERM GOALS

            Use numbers, timeframes, and be medically accurate.
            """

            url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?key={self.gemini_api_key}"
            data = {
                "contents": [{"parts": [{"text": prompt}]}],
                "generationConfig": {"temperature": 0.7, "maxOutputTokens": 1000}
            }
            res = requests.post(url, headers={'Content-Type': 'application/json'}, json=data, timeout=30)

            if res.status_code == 200:
                return res.json()['candidates'][0]['content']['parts'][0]['text']
            return "⚠️ Unable to generate recommendations."

        except Exception as e:
            return f"⚠️ Error: {e}"

    def display_results(self, prediction, probability, user_data, gender):
        """Display prediction and recommendations"""
        print("\n" + "="*50)
        print("PREDICTION RESULTS")
        print("="*50)

        risk = "HIGH RISK" if prediction == 1 else "LOW RISK"
        conf = max(probability) * 100
        prob = probability[1] * 100

        if COLOR:
            print(Fore.RED if prediction else Fore.GREEN + f"\nDiabetes Risk: {risk}")
            print(Fore.YELLOW + f"Confidence: {conf:.2f}%")
            print(Fore.CYAN + f"Diabetes Probability: {prob:.2f}%")
        else:
            print(f"\nDiabetes Risk: {risk}")
            print(f"Confidence: {conf:.2f}%")
            print(f"Diabetes Probability: {prob:.2f}%")

        print("\n" + "="*50)
        print("🤖 AI LIFESTYLE RECOMMENDATIONS")
        print("="*50)

        print("Generating personalized recommendations...")
        tips = self.get_ai_recommendations(user_data, gender, prediction, probability)
        print(tips)

    def run(self):
        """Main loop"""
        self.load_and_train_model()
        while True:
            try:
                user_data, gender = self.get_user_input()
                prediction, probability = self.make_prediction(user_data)
                self.display_results(prediction, probability, user_data, gender)
                again = input("\nWould you like to try another? (y/n): ").lower()
                if again != 'y':
                    print("✅ Thank you for using the system.")
                    break
            except KeyboardInterrupt:
                print("\n👋 Goodbye!")
                break
            except Exception as e:
                print(f"[ERROR] {e}")

# Run
if __name__ == "__main__":
    DiabetesPredictionSystem(csv_path="/content/diabetes.csv").run()


[INFO] Loading data and training model...
[INFO] Model trained with accuracy: 0.846

DIABETES PREDICTION SYSTEM
Enter gender (M/F): m
Glucose level (mg/dL): 120
Blood Pressure (mm Hg): 85
Skin Thickness (mm): 25
Insulin level (mu U/ml): 80
BMI (kg/m²): 28.5
Diabetes Pedigree Function: 0.65
Age (years): 20

PREDICTION RESULTS

Diabetes Risk: LOW RISK
Confidence: 97.00%
Diabetes Probability: 3.00%

🤖 AI LIFESTYLE RECOMMENDATIONS
Generating personalized recommendations...
Okay, here are detailed recommendations for this patient based on the provided profile, even though the risk is currently assessed as low. It's important to reinforce healthy habits early to prevent future development of diabetes.

**Patient Profile Summary:**

*   **Gender:** Male
*   **Age:** 20
*   **BMI:** 28.5 (Overweight)
*   **Glucose:** 120 mg/dL (Prediabetic Range)
*   **Blood Pressure:** 85/ (Normal)
*   **Insulin:** 80 µU/mL (Potentially Elevated - needs context)
*   **Pregnancies:** 0
*   **Diabetes Risk:** L